In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.measure import label
from tqdm import tqdm
from multiprocess import Pool
import os
import warnings

# ============ USER SETTINGS ============
country = "Germany"
region = "MC"
max_pool = 28
domain_area = 200.0 * 200.0  # fixed domain area in km2
pr_th_list = [0.5, 1, 2, 3, 4, 5]
# ========================================

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=RuntimeWarning)

ds_prcp = xr.open_dataset(f"/scratch/nf33/hk25_LSP/{country}/{region}/pr_hourly.nc")
ds_coarse = xr.open_dataset(f"/scratch/nf33/hk25_LSP/{country}/{region}_chunk/rh500_2deg.nc")

lat_centers = ds_coarse.latitude.values
lon_centers = ds_coarse.longitude.values

lat_edges = np.concatenate((lat_centers - 1, [lat_centers[-1] + 1]))
lon_edges = np.concatenate((lon_centers - 1, [lon_centers[-1] + 1]))

for pr_th in pr_th_list:
    def get_sub_prcp(args):
        i, j = args
        num_obj_arr = np.zeros(len(ds_prcp.time))
        tot_area_arr = np.zeros(len(ds_prcp.time))
        mean_obj_area_arr = np.zeros(len(ds_prcp.time))
        area_frac_arr = np.zeros(len(ds_prcp.time))
        cvt_mean_prcp_arr = np.zeros(len(ds_prcp.time))
        tot_mean_prcp_arr = np.zeros(len(ds_prcp.time))
        cvt_tot_prcp_arr = np.zeros(len(ds_prcp.time))

        lat0, lat1 = lat_edges[i], lat_edges[i+1]
        lon0, lon1 = lon_edges[j], lon_edges[j+1]

        sub = ds_prcp.sel(latitude=slice(lat0, lat1), longitude=slice(lon0, lon1))

        for its in range(len(ds_prcp.time)):
            prcp = sub["pr"].isel(time=its) * 3600  # to mm/h
            cvt_prcp = np.sum(prcp.values[prcp.values >= pr_th])
            cv_obj = prcp.copy().fillna(0)
            cv_obj.values[cv_obj.values < pr_th] = 0
            cv_obj.values[cv_obj.values >= pr_th] = 6
            label_arr = label(cv_obj)
            unique_label = np.unique(label_arr)
            num_obj_arr[its] = len(unique_label) - 1

            ind_obj_area = np.zeros(len(unique_label) - 1)
            for ilb in unique_label:
                if ilb == 0:
                    continue
                ind_obj_area[ilb - 1] = np.sum(label_arr == ilb) * 16.0  # 0.04 deg grid ~ 16 km2

            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", category=RuntimeWarning)
                tot_area_arr[its] = np.sum(ind_obj_area)
                if num_obj_arr[its] > 0:
                    mean_obj_area_arr[its] = tot_area_arr[its] / num_obj_arr[its]
                else:
                    mean_obj_area_arr[its] = np.nan
                area_frac_arr[its] = tot_area_arr[its] / domain_area
                if tot_area_arr[its] > 0:
                    cvt_mean_prcp_arr[its] = cvt_prcp / tot_area_arr[its]
                else:
                    cvt_mean_prcp_arr[its] = np.nan
                tot_mean_prcp_arr[its] = cvt_prcp / domain_area
                cvt_tot_prcp_arr[its] = cvt_prcp

        return num_obj_arr, tot_area_arr, mean_obj_area_arr, area_frac_arr, cvt_mean_prcp_arr, tot_mean_prcp_arr, cvt_tot_prcp_arr

    # Initialize arrays
    shape = (len(ds_prcp.time), len(ds_coarse.latitude), len(ds_coarse.longitude))
    num_obj_3d = np.zeros(shape)
    tot_area_3d = np.zeros(shape)
    mean_obj_area_3d = np.zeros(shape)
    area_frac_3d = np.zeros(shape)
    cvt_mean_prcp_3d = np.zeros(shape)
    tot_mean_prcp_3d = np.zeros(shape)
    cvt_tot_prcp_3d = np.zeros(shape)

    args_list = [(i, j) for i in range(len(lat_edges) - 1) for j in range(len(lon_edges) - 1)]

    # Run in parallel
    with Pool(max_pool) as p:
        pool_outputs = list(
            tqdm(
                p.imap(get_sub_prcp, args_list),
                total=len(args_list),
                position=0, leave=True,
                desc=f"pr_th={pr_th} mm/h"
            )
        )

    # Combine outputs
    nlon = len(lon_edges) - 1
    for flat_idx, value in enumerate(pool_outputs):
        i = flat_idx // nlon
        j = flat_idx % nlon
        num_obj_3d[:, i, j], tot_area_3d[:, i, j], mean_obj_area_3d[:, i, j], area_frac_3d[:, i, j], cvt_mean_prcp_3d[:, i, j], tot_mean_prcp_3d[:, i, j], cvt_tot_prcp_3d[:, i, j] = value

    # Save output
    output_path = f"/scratch/nf33/hk25_LSP/{country}/{region}_chunk/number_size_prth_{pr_th}_hourly.nc"
    out = xr.Dataset(
        {
            "num_obj": (("time", "latitude", "longitude"), num_obj_3d),
            "tot_area": (("time", "latitude", "longitude"), tot_area_3d),
            "mean_obj_area": (("time", "latitude", "longitude"), mean_obj_area_3d),
            "area_frac": (("time", "latitude", "longitude"), area_frac_3d),
            "cvt_mean_prcp": (("time", "latitude", "longitude"), cvt_mean_prcp_3d),
            "tot_mean_prcp": (("time", "latitude", "longitude"), tot_mean_prcp_3d),
            "cvt_tot_prcp": (("time", "latitude", "longitude"), cvt_tot_prcp_3d),
        },
        coords={
            "time": ds_prcp.time.values,
            "latitude": ds_coarse.latitude.values,
            "longitude": ds_coarse.longitude.values,
        },
    )
    out.to_netcdf(output_path)
    print(f"Saved to {output_path}")
    out.close()

ds_prcp.close()


pr_th=0.5 mm/h: 100%|██████████| 325/325 [03:41<00:00,  1.47it/s]


Saved to /scratch/nf33/hk25_LSP/Germany/MC_chunk/number_size_prth_0.5_hourly.nc


pr_th=1 mm/h: 100%|██████████| 325/325 [02:44<00:00,  1.97it/s]


Saved to /scratch/nf33/hk25_LSP/Germany/MC_chunk/number_size_prth_1_hourly.nc


pr_th=2 mm/h: 100%|██████████| 325/325 [02:52<00:00,  1.89it/s]


...

Saved to /scratch/nf33/hk25_LSP/Germany/MC_chunk/number_size_prth_3_hourly.nc


pr_th=4 mm/h: 100%|██████████| 325/325 [02:42<00:00,  2.00it/s]


Saved to /scratch/nf33/hk25_LSP/Germany/MC_chunk/number_size_prth_4_hourly.nc


pr_th=5 mm/h: 100%|██████████| 325/325 [02:37<00:00,  2.06it/s]


Saved to /scratch/nf33/hk25_LSP/Germany/MC_chunk/number_size_prth_5_hourly.nc
